In [102]:
import numpy as np
import torch
from scipy.stats import multivariate_normal
from OGMM import OGMM, init_cov_and_T_split
from sklearn.mixture import GaussianMixture
from scipy.special import logsumexp

In [103]:
dataset = np.load('MGR.npy')
domain_names = ['Noiseless', "buccaneer2", "destroyerengine", "f16", "factory2"]

In [104]:
data_noiseless = dataset[:1000, :]
# feature column 17 is only zeros
data_noiseless = np.delete(data_noiseless, 17, axis=1)

# separation between training and validation data
data_noiseless = np.random.permutation(data_noiseless)
prop_train = 0.8
data_train = data_noiseless[:int(prop_train * len(data_noiseless)), :]
data_test = data_noiseless[int(prop_train * len(data_noiseless)):, :]


# Features of the noiseless data
X_train = data_train[:, :-2]
X_train_t = torch.from_numpy(X_train)
X_test = data_test[:, :-2]
X_test_t = torch.from_numpy(X_test)

# Labels of the noiseless data
y_train = data_train[:, -2]
y_test = data_test[:, -2]

Log likelihood function

In [105]:
GMM_sklearn = GaussianMixture(n_components=10, random_state=0).fit(X_train)

In [106]:
np.sum(GMM_sklearn.score_samples(X_test))

-46555.67281112568

In [107]:
def normal_pdf(x, mean, cov, bib='numpy'):
    if bib == 'numpy':
        return (2*np.pi)**(-len(x)/2) * np.linalg.det(cov)**(-1/2) * np.exp(-1/2 * (x - mean).T @ np.linalg.inv(cov) @ (x - mean))
    else:
        return (2*torch.pi)**(-len(x)/2) * torch.linalg.det(cov)**(-1/2) * torch.exp(-1/2 * (x - mean).T @ torch.inverse(cov) @ (x - mean))

def log_normal_pdf(x, mean, cov, bib='numpy'):
    if bib == 'numpy':
        return -len(x)/2*np.log(2*np.pi) - 1/2*(np.log(np.linalg.det(cov)) + (x - mean).T @ np.linalg.inv(cov) @ (x - mean))
    else:
        return -len(x)/2*torch.log(torch.tensor(2*torch.pi)) - 1/2*(torch.log(torch.linalg.det(cov)) + (x - mean).T @ torch.inverse(cov) @ (x - mean))

In [108]:
def log_likelihood(mixture, X, bib='numpy'):
    log_pdf = np.zeros((X.shape[0], len(mixture)))
    for i in range(X.shape[0]):
        for j in range(len(mixture)):
            log_pdf[i, j] = np.log(mixture[j][0]) + log_normal_pdf(X[i], mixture[j][2], mixture[j][3], bib)
            
    return np.sum(logsumexp(log_pdf, axis=1))

In [109]:
means = GMM_sklearn.means_
weights = GMM_sklearn.weights_
cov = GMM_sklearn.covariances_
GMM_sklearn_list = [[weights[i], 1, means[i], cov[i]] for i in range(len(weights))]

In [110]:
ll_sklearn = log_likelihood(GMM_sklearn_list, X_test)

ll_sklearn

-46555.67281112571

In [111]:
range_data = np.mean(np.max(X_train, axis=0) - np.min(X_train, axis=0))

K_max = 10
cov_init, T_split = init_cov_and_T_split(K_max, range_data, X_train.shape[1])
GMM_online = OGMM(X_train_t, K_max, cov_init, 0.1, T_split)

C:\Users\SL276123\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\linalg\linalg.py:2140: RuntimeWarning: overflow encountered in cast
  r = r.astype(result_t, copy=False)


In [112]:
GMM_online

[[tensor(0.1968),
  tensor(100.),
  tensor([ 3.8876e-01,  8.1317e-02,  1.3761e-01,  1.4987e-03,  2.1281e+03,
           1.4574e+05,  2.0349e+03,  5.3696e+04,  4.2419e+03,  5.0994e+05,
           1.1424e-01,  1.3558e-03, -6.0637e-04,  1.5595e-02, -1.0736e-03,
           4.6675e-03,  1.2641e+02, -1.4307e+02,  2.8205e+03,  1.0452e+02,
           4.2817e+02, -2.3443e+01,  3.1020e+02,  4.4183e+01,  1.3069e+02,
          -8.2713e+00,  1.1397e+02,  1.7819e+01,  7.5391e+01, -9.7840e+00,
           8.1390e+01,  1.1658e+01,  6.3727e+01, -9.4091e+00,  7.4993e+01,
           9.3854e+00,  6.3229e+01, -8.1605e+00,  6.2102e+01,  6.3470e+00,
           6.3686e+01, -5.7481e+00,  6.6625e+01,  4.1401e+00,  5.8578e+01,
          -3.4956e+00,  5.1413e+01,  4.3524e+00,  5.5746e+01, -2.5098e+00,
           4.5319e+01,  1.6462e+00,  5.3699e+01, -2.4740e+00,  6.5222e+01,
           4.9931e-01,  6.3517e+01], dtype=torch.float64),
  tensor([[ 1.5287e-02, -8.2705e-04,  4.8666e-03,  ..., -3.4190e+00,
            1

In [113]:
ll_online = log_likelihood(GMM_online, X_test_t, 'torch')
ll_online

-142515.61475152257

In [114]:
(ll_online - ll_sklearn)/ll_sklearn

2.0611868789803998